In [1]:
import warnings
warnings.filterwarnings('ignore')

# Test whisper models

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print(f"device: {device}")

def initialize_pipe(model, model_path):
    
    # model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True, attn_implementation="flash_attention_2") # use_flash_attention_2=True) # 
    # model.to(device)

    if hasattr(model.generation_config, "no_timestamps_token_id"):
        return_timestamps = True
    else:
        return_timestamps = False
        
    processor = AutoProcessor.from_pretrained(model_path)

    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        max_new_tokens=128,
        chunk_length_s=30,
        batch_size=16,
        return_timestamps=return_timestamps,
        torch_dtype=torch_dtype,
        device=device,
    )
    
    return pipe

device: cuda:0


# Test and compare different versions

## Common voice 17.0, zh-HK

In [ ]:
# import os
# import pandas as pd

# dataset_name = "common_voice"

# test_data_dir = "data/cv-corpus-17.0-2024-03-15/zh-HK/clips"
# test_data_path = "data/cv-corpus-17.0-2024-03-15/zh-HK/test.tsv"

# df = pd.read_csv(test_data_path, sep='\t')
# print(df.shape)
# df.head()

# num_test = 1000

# df.rename(columns={"path": "file_id", "sentence": "true"}, inplace=True)
# df = df[:num_test]
# df.head()

## midea data

In [ ]:
# import pandas as pd
# dataset_name = 'midea_500'
# # test_data_dir = "data/midea_data_500/clips"
# # test_data_dir = "data/midea_data_500/amrs"
# test_data_dir = "data/midea_data_500/wavs"

# df = pd.read_csv("./data/midea_data_500/transcripts.csv")
# df.rename(columns={"sentence": "true"}, inplace=True)
# # df['file_id'] = df['file_id'].apply(lambda x: x.replace('.mp3', '.amr'))
# df.head()

# custom_data_v0

In [ ]:
# import pandas as pd

# dataset_name = 'custom_v0'
# test_data_dir = "data/custom_data_v0/test"

# with open(os.path.join(test_data_dir, 'audio_paths')) as fo:
#     audio_paths = fo.readlines()
    
# with open(os.path.join(test_data_dir, 'text')) as fo:
#     texts = fo.readlines()
    
# print(audio_paths[:5])
# print(texts[:5])

# start_id = 0
# df = pd.DataFrame([[text.strip().split()[0], " ".join(text.strip().split()[1:])] for text in texts[start_id:]], columns=['idx', 'true'])
# df.head()

# df_path = pd.DataFrame([path.strip().split() for path in audio_paths[start_id:]], columns=['idx', 'file_id'])
# df_path.head()

# df = pd.merge(df, df_path, on='idx')
# df.head()

# test_20240816

In [13]:
import pandas as pd

dataset_name = 'test_0816'
test_path = "data/test_0816/ai辅助填单数据/模型返回的内容.csv"
wav_dir = "data/test_0816/ai辅助填单数据/录音文件"

df = pd.read_csv(test_path)
# start_id = 0
# df = pd.DataFrame([[text.strip().split()[0], " ".join(text.strip().split()[1:])] for text in texts[start_id:]], columns=['idx', 'true'])
# df.head()

# df_path = pd.DataFrame([path.strip().split() for path in audio_paths[start_id:]], columns=['idx', 'file_id'])
# df_path.head()

# df = pd.merge(df, df_path, on='idx')
df.head()

,contact_id,content,extracted_content
0,0c6c1fda-105d-43aa-bb5d-25486b3ed249,customer:可疑唔該做呀\nagent:喂你好門\nagent:喂你好門公司服務部係唔...,"_______query_SR__costTime:68542ms;{""statusCode..."
1,18522785-539a-4594-9ac6-1062d0f686a2,"agent:好,公司合作,你幫\ncustomer:係咪東支嗰邊呀\nagent:無錯,有咩...","_______query_SR__costTime:15592ms;{""statusCode..."
2,4028279d-fe0d-4898-a079-4a63e1cc16c2,agent:早晨通知服務鋪外出糧請買咩幫你�哎�\ncustomer:我姓楊㗎\ncusto...,"_______query_SR__costTime:8293ms;{""statusCode""..."
3,930224b6-17d3-4e8f-9da3-9878da6f9a10,agent:回到你\nagent:你好\ncustomer:喂你好呀我之前\nagent:係...,"_______query_SR__costTime:6647ms;{""statusCode""..."
4,1933fbf7-e0c0-4d50-abe7-d5e137545ee6,agent:早晨�早晨�總之復活部�我姓梁�請問咩幫你�\ncustomer:你好我想預約呢...,"_______query_SR__costTime:6899ms;{""statusCode""..."


## chunk 10s

In [26]:
import pandas as pd
import glob
import os

dataset_name = 'test_0816'
wav_dir = "data/test_0816/short_10s"

wav_files = sorted(glob.glob(os.path.join(wav_dir, '*.wav')))

df = pd.DataFrame(wav_files, columns=['contact_id'])
# start_id = 0
# df = pd.DataFrame([[text.strip().split()[0], " ".join(text.strip().split()[1:])] for text in texts[start_id:]], columns=['idx', 'true'])
# df.head()

# df_path = pd.DataFrame([path.strip().split() for path in audio_paths[start_id:]], columns=['idx', 'file_id'])
# df_path.head()

# df = pd.merge(df, df_path, on='idx')
df.head()

,contact_id
0,data/test_0816/short_10s/04d9e7e2-b06d-48de-8a...
1,data/test_0816/short_10s/04d9e7e2-b06d-48de-8a...
2,data/test_0816/short_10s/04d9e7e2-b06d-48de-8a...
3,data/test_0816/short_10s/04d9e7e2-b06d-48de-8a...
4,data/test_0816/short_10s/04d9e7e2-b06d-48de-8a...


## clear GPU

In [14]:
import gc

# Clear the model from GPU memory
def clear_model_from_gpu(model):
    # Delete the model object
    del model
    # Force garbage collection to free any unreferenced memory
    gc.collect()
    # Clear any existing CUDA cache
    torch.cuda.empty_cache()
    # Optionally, you can reinitialize the CUDA context by calling CUDA operations
    torch.cuda.synchronize()
    
    print("Model cleared from GPU memory.")

## convert a checkpoint to faster whisper

In [20]:
# ckpt_dir = "/home/ec2-user/SageMaker/efs/Projects/whisper/checkpoint/checkpoint-v7/checkpoint-60"

# save_dir = "./model"

In [21]:
# !ct2-transformers-converter --model {ckpt_dir} --output_dir {save_dir} \
# --copy_files tokenizer.json preprocessor_config.json --quantization float16

In [22]:
import os
import json
import time
from tqdm import tqdm
from faster_whisper import WhisperModel
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

# Here, change the checkpoint path and the whisper model path

model_ids = [
    # {"label": "v0-60", "model_path": "/home/ec2-user/SageMaker/efs/Projects/whisper/checkpoint/checkpoint-v0/checkpoint-60"},
    {"label": "v7-60", "model_path": "/home/ec2-user/SageMaker/efs/Projects/whisper/checkpoint/checkpoint-v7/checkpoint-60"},
    {"label": "faster-v7-60", "model_path": "/home/ec2-user/SageMaker/efs/Projects/whisper/checkpoint/checkpoint-v7/checkpoint-60-faster"},  
    # {"label": "faster-v0-60", "model_path": "/home/ec2-user/SageMaker/efs/Projects/whisper/checkpoint/checkpoint-v0-bs64/checkpoint-60-faster"},  
    {"label": "large-v3", "model_path": "/home/ec2-user/SageMaker/efs/Models/whisper-large-v3"},
    {"label": "faster-large-v3", "model_path": "/home/ec2-user/SageMaker/efs/Models/faster-whisper-large-v3"}, 
]

sample = os.path.join(wav_dir, df.loc[0, 'contact_id']+'.wav')

results = []

num_test = 50
df = df[:num_test]

for model_id in model_ids:
    label = model_id["label"]
    model_path = model_id["model_path"]
    
    if 'faster' in label: 
        model = WhisperModel(model_path)
        # model.to(device)
        segments, info = model.transcribe(sample, language="yue") # 
        pred = ''.join([segment.text for segment in segments])
        print(f"model label: {label}, result: {pred}")
    else:
        model = AutoModelForSpeechSeq2Seq.from_pretrained(model_path, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True, attn_implementation="flash_attention_2")
        model.to(device)
        
        pipe = initialize_pipe(model, model_path)
        if label == "large-v3" or label.startswith("ckpt"):
            result = pipe(sample, generate_kwargs={"language": "cantonese"})
        else:
            result = pipe(sample)
            
        print(f"model label: {label}, result: {result['text']}")
    
    for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
        filename = row['contact_id'] + '.wav'
        filepath = os.path.join(wav_dir, filename)
        
        if not os.path.exists(filepath):
            continue
        
        start_time = time.time()
        if label == "large-v3" or label.startswith("ckpt"):
            result = pipe(filepath, generate_kwargs={"language": "cantonese"})
        elif label == "faster-large-v3":
            segments, info = model.transcribe(filepath, language="yue") # 
            # for segment in segments:
                # print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
            result = {'text': ''.join([segment.text for segment in segments])}
        else:
            result = pipe(filepath)
        
        time_cost = time.time() - start_time
        
        df.loc[idx, f"pred-{label}"] = result['text']
        df.loc[idx, f"time-{label}"] = round(time_cost, 2)
        df.loc[idx, f"chunks-{label}"] = json.dumps(result.get("chunks", ""))
    
    clear_model_from_gpu(model)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.73it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model label: v7-60, result: 唔解唔解喂你好mon喂你好mon東芝服務部係唔幫到你㗎呢係咁樣㗎我想問一問哦喂係唔幫到你㗎呢我嗰個請緊我個焗爐壞咗啊麻煩你搵人過去整啦嗱小姐你嗰個焗爐嗰個型號麻煩你咩提供畀我唔該我畀我電話畀你囉好無問題係98827273好你我想問一問我想問一個問題小姐你係咪之前裝過公司續保嚟㗎係係係呀好得等等點稱呼嘅呢小姐本人我姓王嘅好等開陣喇王小姐等等六八二七二七三係喇將佢怒心都成一期錄咗三十七流嘅處我我想問一問嘅王小姐依家你報徵過路佢咩問題㗎呃我唔知佢個喉塞咗喇去即係總之就成日都F02囉但係你當時嘅水盒係滿曬水㗎嘛係咪去喎去喎去喎去喎有冇試過將嗰個水盒即係成個水盒拉出去再拍返圓位佢會唔會再出一份嘢都唔得唔得�你等多一陣喇咁就等一等將軍我信到成果比啦多望望如果最快呢我哋呢邊就可以安排師傅七月廿三號上門唔知廿七我們方唔方便王先生有咩問時間嗰度呢你預佢我哋師傅上門時間都係廿三號當日大約中午十二點至下集五點鐘呢段時間咁師傅黎之前呢我估佢會再畀個電話畀你先嘅我邊台照落單啦王小姐喺呢度好唔該曬你呢度唔該啦拜拜好唔該你拜拜


100%|██████████| 21/21 [02:23<00:00,  6.85s/it]


Model cleared from GPU memory.
model label: faster-v7-60, result: 唔解唔解喂你好mon喂你好mon東芝服務部係唔幫到你㗎呢係咁樣㗎我唔想咁樣喂係唔幫到你㗎呢點解我個焗爐壞咗呀麻煩你搵人過去整啦嗱小姐你嗰個焗爐嗰個型號麻煩你咩提供畀我唔該請個個型號我畀我個電話畀你喎好無問題係呀係九八八二七二七三好你喇我想問一問我想問一個問題小姐你唔係之前裝過公司續保嚟㗎係係係呀好得等等點稱呼嘅呢小姐本人我姓王㗎好等開陣喇王小姐等等九八八二七二七三係喇將會路伸到成一期六座三十七樓的船哦我想問一問嘅王小姐依家你報徵過路佢咩問題㗎呃我唔知佢個喉塞咗喇去即係總之就成日都F02囉但係你當時嘅水盒係滿曬水㗎嘛係咪去喎去喇去喇去喇有冇試過將嗰個水盒即成個水盒拉出嚟再拍返圓位佢會唔會再展出F2都唔得唔得唔明你等多一陣喇咁就等一等將功勞伸到成嗰邊喇咁望唔望如果最快呢我哋呢邊就可以安排師傅七月廿三號上門唔知廿七我哋方唔方便王小姐想問問時間嗰度呢你預佢我哋師傅上門時間都係廿三號當日大約中午十二點至下午五點鐘呢段時間咁師傅嚟之前呢我佢都會再畀個電話畀你先嘅我邊台照落單喇王小姐喺呢度好唔該曬你呢度唔該拜拜好唔該你拜拜好唔該你拜拜


100%|██████████| 21/21 [02:24<00:00,  6.87s/it]


...

model label: large-v3, result:  喂,你好,Mone,東芝服務部,係咪幫到你㗎呢? 喂?係咪幫到你㗎呢? 喂,請問我個焗爐壞咗,麻煩你搵人過去整啦 嗱,小姐,你嗰個焗爐嗰個型號麻煩你咩�局奴的型號 我給你電話號碼 98827273 我想問一個問題 你是不是之前加入過的公司續保? 是的 怎樣稱呼的呢? 我姓黃既。 好,等開一陣啦,黃小姐,等一等。 八八二七二七三, 係喇,將我路伸到成一期六座三十七樓一處。 哦,我想問一問既黃小姐,依家你報徵過路佢咩問題架? 呃,我唔知係咪個喉塞左 總之就成日都F02 但係你當時水盒係滿曬水架嘛係咪 係喎係喎係喎 有冇試過將個水盒 即係成個水盒拉出嚟再拍返原位 佢會唔會再減出F02 都唔得 唔得 將官奧信到成果備 再看看 如果最快我們這邊就可以安排師傅 7月23日上門 不知道23日會否方便 黃先生有什麼問題 時間那裡你預計師傅上門時間 都是23日當天大約 中午12點至下午5點鐘 呢段時間,師傅來之前呢 都會再給個電話給你先的 我一邊同你照落單啦,黃小姐,好,謝謝你,拜拜 好,謝謝你,拜拜


100%|██████████| 21/21 [02:08<00:00,  6.11s/it]


Model cleared from GPU memory.
model label: faster-large-v3, result:  喂,你好Mon,東芝服務部,係咪幫到你㗎呢? 喂?係咪幫到你㗎呢? 小姐,你嗰個焗爐嗰個型號麻煩你咩提供畀我唔該? 我畀我電話畀你喎,因為... 無問題 係,98827273 我想問一個問題,小姐你係咪之前join過個公司族補嚟㗎? 係係係呀 好,得,等,點稱呼既呢?小姐本人 我姓黃㗎 好,等開一陣啦,黃小姐等 98827273 係喇,張安路,新都城,一期六座,三十七樓 係喇,張安路,新都城,一期六座,三十七樓 係喇,張安路,新都城,一期六座,三十七樓 嘩,我想問一問嘅,黃小姐,依家你報徵過路佢咩問題㗎? 唔知咪個喉塞咗呀佢,總之就成日都F02囉 但係你當時水盒係滿曬水㗎嘛係咪? 係喎係喎係喎係喎 有冇試過將個水盒,即係成個水盒拉出嚟再拍返原位,佢會唔會再... 有呀有呀 都唔得? 唔得 唔明,你等多一陣啦,咁就等一等 張安路,新都城,那邊,我睇睇 如果最快呢,我哋呢邊就可以安排師傅七月廿三號上門 唔知廿三號你方唔方便,黃小姐想問一問 時間嗰度呢,你預佢我哋師傅上門時間都係廿三號當日 大約中午十二點至下午五點鐘呢段時間 咁師傅嚟之前呢,我都會再比個電話畀你先嘅 我一邊同你照落單啦,黃小姐,你都 好,唔該了 拜拜 唔該了,拜拜 拜拜


100%|██████████| 21/21 [09:07<00:00, 26.06s/it]


Model cleared from GPU memory.


In [23]:
df.head()

,contact_id,content,extracted_content,pred-v7-60,time-v7-60,chunks-v7-60,pred-large-v3,time-large-v3,chunks-large-v3,pred-faster-large-v3,time-faster-large-v3,chunks-faster-large-v3,pred-faster-v7-60,time-faster-v7-60,chunks-faster-v7-60
0,0c6c1fda-105d-43aa-bb5d-25486b3ed249,customer:可疑唔該做呀\nagent:喂你好門\nagent:喂你好門公司服務部係唔...,"_______query_SR__costTime:68542ms;{""statusCode...",唔解唔解喂你好mon喂你好mon東芝服務部係唔幫到你㗎呢係咁樣㗎我想問一問哦喂係唔幫到你㗎呢...,4.23,"[{""timestamp"": [0.0, 1.0], ""text"": ""\u5514\u89...","喂,你好,Mone,東芝服務部,係咪幫到你㗎呢? 喂?係咪幫到你㗎呢? 喂,請問我個焗爐壞...",3.76,"[{""timestamp"": [0.0, 7.0], ""text"": "" \u5582,\u...","喂,你好Mon,東芝服務部,係咪幫到你㗎呢? 喂?係咪幫到你㗎呢? 小姐,你嗰個焗爐嗰個型...",10.44,"""""",唔解唔解喂你好mon喂你好mon東芝服務部係唔幫到你㗎呢係咁樣㗎我想問一問哦喂係唔幫到你㗎呢...,4.31,"[{""timestamp"": [0.0, 1.0], ""text"": ""\u5514\u89..."
1,18522785-539a-4594-9ac6-1062d0f686a2,"agent:好,公司合作,你幫\ncustomer:係咪東支嗰邊呀\nagent:無錯,有咩...","_______query_SR__costTime:15592ms;{""statusCode...",東支那邊呀無錯我諗得幫到你你好呀我去屋企嗰部即係個洗衣機唔知咩事呢洗洗客衫咩突然呢導致團屋跳...,4.05,"[{""timestamp"": [0.0, 3.4], ""text"": ""\u6771\u65...","東芝那邊 沒錯,我想到你 你好,我家裡的洗衣機不知道什麼事 洗衣服突然導致屯屋跳電 跳災,...",3.75,"[{""timestamp"": [0.0, 3.6], ""text"": "" \u6771\u8...",東芝那邊 沒錯 我想幫到你 你好 我家裏的洗衣機不知甚麼事 洗衣服突然導致屯屋跳電 跳債是...,8.02,"""""",東支那邊呀無錯我諗得幫到你你好呀我去屋企嗰部即係個洗衣機唔知咩事呢洗洗客衫咩突然呢導致團屋跳...,4.05,"[{""timestamp"": [0.0, 3.4], ""text"": ""\u6771\u65..."
2,4028279d-fe0d-4898-a079-4a63e1cc16c2,agent:早晨通知服務鋪外出糧請買咩幫你�哎�\ncustomer:我姓楊㗎\ncusto...,"_______query_SR__costTime:8293ms;{""statusCode""...",早晨通知服務部外送娘整咩咩幫你我姓楊架係係我呢發覺我個通知嘅洗衣機呢有啲問題呀嗯有啲大約咩問...,9.84,"[{""timestamp"": [0.0, 4.86], ""text"": ""\u65e9\u6...","早晨,東芝服務部,我係梁靜雯,有咩幫你? 我係姓楊,我發覺東芝洗衣機有啲問題 有啲大約咩問...",8.75,"[{""timestamp"": [0.0, 5.0], ""text"": "" \u65e9\u6...","早晨通知服務部,我姓梁,請問有什麼幫你? 我姓楊,我發覺我通知的洗衣機有些問題 有些大約什...",29.47,"""""",早晨通知服務部外送娘整咩咩幫你我姓楊架係係我呢發覺我個通知嘅洗衣機呢有啲問題呀嗯有啲大約咩問...,9.85,"[{""timestamp"": [0.0, 4.86], ""text"": ""\u65e9\u6..."
3,930224b6-17d3-4e8f-9da3-9878da6f9a10,agent:回到你\nagent:你好\ncustomer:喂你好呀我之前\nagent:係...,"_______query_SR__costTime:6647ms;{""statusCode""...",搬到你喂你好呀你好之前係呀之前安排咗師傅上去我度睇嘅睇完之後我會搵運輸搵走但到依家都未有se...,4.77,"[{""timestamp"": [0.0, 1.0], ""text"": ""\u642c\u52...",幫到你 喂你好呀 之前安排咗師傅上去我度睇嘅 睇完之後我以為會運輸運走 但到依家都未有聲氣...,4.22,"[{""timestamp"": [0.0, 1.0], ""text"": "" \u5e6b\u5...","你好呀,之前安排咗師傅上去我度睇嘅,睇完之後會搵書搵酒,但到依家都未有聲氣,因為我之後要去...",10.86,"""""",搬到你喂你好呀你好之前係呀之前安排咗師傅上去我度睇嘅睇完之後我會搵運輸搵走但到依家都未有se...,4.64,"[{""timestamp"": [0.0, 1.0], ""text"": ""\u642c\u52..."
4,1933fbf7-e0c0-4d50-abe7-d5e137545ee6,agent:早晨�早晨�總之復活部�我姓梁�請問咩幫你�\ncustomer:你好我想預約呢...,"_______query_SR__costTime:6899ms;{""statusCode""...",早晨早晨總之復活部我姓梁請問咩幫你你好我想預約呢洗機維修㗎但應該就過咗個唔小姐有冇電話號碼可...,4.61,"[{""timestamp"": [0.0, 4.8], ""text"": ""\u65e9\u66...","早晨,中資庫務部,我姓梁,請問有什麼幫你? 你好,我想約洗衣機維修,但應該過了個... 嗯...",4.13,"[{""timestamp"": [0.0, 5.0], ""text"": "" \u65e9\u6...","早晨,中資服務部,我姓梁,請問有什麼幫你? 你好,我想約洗衣機維修,但應該過了個… 嗯,小...",11.46,"""""",早晨早晨總之復活部我姓梁請問咩幫你你好我想預約呢洗機維修㗎但應該就過咗個唔小姐有冇電話號碼可...,4.60,"[{""timestamp"": [0.0, 4.8], ""text"": ""\u65e9\u66..."


In [24]:
df.to_csv("data/test_0816/comparisons.csv", index=False)

## CER calculation

In [ ]:
import re
import jiwer
from whisper.normalizers import BasicTextNormalizer, EnglishTextNormalizer

class ChineseTextNormalizer:
    def __init__(self, split_letters: bool = False):

        self.split_letters = split_letters

    def __call__(self, s: str):
        s = s.lower()
        # s = re.sub(r"[<\[][^>\]]*[>\]]", "", s)  # remove words between brackets
        # s = re.sub(r"\(([^)]+?)\)", "", s)  # remove words between parenthesis
        # s = self.clean(s).lower()

        # if self.split_letters:
        #     s = " ".join(regex.findall(r"\X", s, regex.U))

        s = re.sub(
            r"\s+", "", s
        )  # remove any successive whitespace characters
        
        return s
    
# normalizer = EnglishTextNormalizer()
# normalizer = BasicTextNormalizer()
normalizer = ChineseTextNormalizer()

In [ ]:
df.head()

In [ ]:
new_columns = ['true', 'true-clean']
pred_cols, cer_cols, time_cols = [], [], []
chunk_cols = []
for model_id in model_ids:
    label = model_id['label']
    pred_cols.extend([f'pred-{label}',f'pred-{label}-clean'])
    cer_cols.append(f'cer-{label}')
    if f'chunks-{label}' in df.columns:
        chunk_cols.append(f'chunks-{label}')
        
    time_cols.append(f'time-{label}')

new_columns.extend(pred_cols+cer_cols+time_cols+chunk_cols)
# new_columns.extend(['file_id', 'user_id', 'category_name'])
# new_columns.extend(['file_id', 'user_id', 'category_name'])
new_columns.extend(['file_id'])


for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    
    true_clean = normalizer(row['true'])
    df.loc[i, 'true-clean'] = true_clean
    
    for model_id in model_ids:
        label = model_id['label']
        pred_clean = normalizer(row[f'pred-{label}'])
        df.loc[i, f'pred-{label}-clean'] = pred_clean
        
        cer = jiwer.cer(true_clean, pred_clean)
        df.loc[i, f'cer-{label}'] = cer
        
df = df[new_columns]
df = df.fillna("")
df.to_csv(f"outputs/transcripts_{dataset_name}.csv", index=False)
df.head(10)

In [ ]:
df = df.fillna("")
cers = []
for model_id in model_ids:
    label = model_id['label']
    df = df[df[f"pred-{label}-clean"]!='']
    cer = jiwer.cer(list(df["true-clean"]), list(df[f"pred-{label}-clean"]))
    
    cers.append({'model': label, 'cer': cer})

df_cers = pd.DataFrame(cers)
df_cers.to_csv(f"outputs/cers_{dataset_name}.csv", index=False)
df_cers.head()

In [ ]:
df_cer_stats = df[cer_cols].describe(percentiles=[0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99])
df_cer_stats.to_csv(f"outputs/cer_stats_{dataset_name}.csv")
df_cer_stats

In [ ]:
df_time_stats = df[time_cols].describe(percentiles=[0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99])
df_time_stats.to_csv(f"outputs/time_stats_{dataset_name}.csv")
df_time_stats

# Calculate the statistics of dataset

In [ ]:
# import pandas as pd

# path = "data/cv-corpus-17.0-2024-03-15/zh-HK/clip_durations.tsv"

# df = pd.read_csv(path, sep='\t')
# df.head()

In [ ]:
# df['duration[ms]'].describe()